## 4 - Network Structure
How many neurons are simulated? 

How many degrees of freedom (weights) does the network have?

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from numpy.random import choice, random_integers
from mnist import MNIST
import tensorflow as tf


data = MNIST()
BATCHSIZE = 100
EPOCHS = 10000
TESTSTEPSIZE = 100

## Model

In [ ]:
def batchnorm(y, is_test, iteration, offset, convolutional=False):
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.9999, iteration)
    mean, variance = tf.nn.moments(y, [0, 1, 2] if convolutional else [0])
    update_moving_avgs = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    ybn = tf.nn.batch_normalization(y, m, v, offset, None, 1e-5)
    return ybn, update_moving_avgs

In [ ]:
x = tf.placeholder(tf.float32, (None, 28, 28), name='input')
t = tf.placeholder(tf.int64, (None), name='target')
lr = tf.placeholder(tf.float32)
it = tf.placeholder(tf.int32)
tst = tf.placeholder(tf.bool)

x_ = tf.reshape(x, (-1,28,28,1))

w0 = tf.Variable(tf.truncated_normal((6,6,1,6), stddev=0.1))
b0 = tf.Variable(tf.ones((6,))/10)
c0 = tf.nn.conv2d(x_, w0, (1,1,1,1), 'SAME')
n0, u0 = batchnorm(c0, tst, it, b0, convolutional=True)
y0 = tf.nn.relu(n0)

w1 = tf.Variable(tf.truncated_normal((5,5,6,12), stddev=0.1))
b1 = tf.Variable(tf.ones((12,))/10)
c1 = tf.nn.conv2d(y0, w1, (1,2,2,1), 'SAME')
n1, u1 = batchnorm(c1, tst, it, b1, convolutional=True)
y1 = tf.nn.relu(n1)

w2 = tf.Variable(tf.truncated_normal((4,4,12,24), stddev=0.1))
b2 = tf.Variable(tf.ones((24,))/10)
c2 = tf.nn.conv2d(y1, w2, (1,2,2,1), 'SAME')
n2, u2 = batchnorm(c2, tst, it, b2, convolutional=True)
y2 = tf.nn.relu(n2)

w3 = tf.Variable(tf.truncated_normal((7*7*24,200), stddev=0.1))
b3 = tf.Variable(tf.ones((200,))/10)
y2_ = tf.reshape(y2, (-1,7*7*24))
n3, u3 = batchnorm(tf.matmul(y2_, w3), tst, it, b3)
r3 = tf.nn.relu(n3)
y3 = tf.nn.dropout(r3, 0.75)

wo = tf.Variable(tf.truncated_normal((200,10), stddev=0.1))
bo = tf.Variable(tf.zeros((10,)))
y = tf.matmul(y3, wo) + bo

update_ema = tf.group(u0, u1, u2, u3)

entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, t)
entropy = tf.reduce_mean(entropy) * 100
train_step = tf.train.AdamOptimizer(lr).minimize(entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), t)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Training

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

rate_max = 0.003
rate_min = 0.0001
rate_decay = 2000

for i in range(1, EPOCHS + 1):
    batch_x, batch_t = data.getTrainingBatch(10)
    rate = rate_min + (rate_max - rate_min) * np.exp(-i/rate_decay)
    
    sess.run(train_step, {x: batch_x, t: batch_t, lr: rate, tst: False})
    sess.run(update_ema, {x: batch_x, t: batch_t, tst: False, it: i})
    if i % 100 == 0:
        acc = sess.run(accuracy, {x: data.testData, t: data.testLabels, tst: True})
        print(i, acc, rate)